In [12]:
import pandas as pd

data = pd.read_excel('./datasets/supermarket_sales.xlsx')
data[['Gender', 'Product line', 'Total']]

,Gender,Product line,Total
0,Female,Health and beauty,548.9715
1,Female,Electronic accessories,80.2200
2,Male,Home and lifestyle,340.5255
3,Male,Health and beauty,489.0480
4,Male,Sports and travel,634.3785
...,...,...,...
995,Male,Health and beauty,42.3675
996,Female,Home and lifestyle,1022.4900
997,Male,Food and beverages,33.4320
998,Male,Home and lifestyle,69.1110


In [13]:
#Pivot table
#A pivot table is a table of grouped values that aggregates the individual items of a more extensive table within one or more 
#discrete categories. This summary might include sums, averages, or other statistics, which the pivot table groups together 
#using a chosen aggregation function applied to the grouped values.

pivot_table = data.pivot_table(index='Gender', columns='Product line', values='Total', aggfunc='sum').round(0)

pivot_table.to_excel('pivot_table.xlsx', 'Report', startrow=4)
pivot_table

Product line,Electronic accessories,Fashion accessories,Food and beverages,Health and beauty,Home and lifestyle,Sports and travel
Gender,,,,,,
Female,27102.0,30437.0,33171.0,18561.0,30037.0,28575.0
Male,27236.0,23868.0,22974.0,30633.0,23825.0,26548.0


In [14]:
%pip install openpyxl

from openpyxl import load_workbook, Workbook
from openpyxl.chart.bar_chart import BarChart
from openpyxl.chart.reference import Reference

wb: Workbook = load_workbook('pivot_table.xlsx')
sheet: Workbook = wb['Report']

min_col: int = wb.active.min_column
max_col: int = wb.active.max_column
min_row: int = wb.active.min_row
max_row: int = wb.active.max_row

barchart = BarChart()

# data is numbers + headers
data_from_sheet = Reference(
    sheet,
    min_col=min_col+1,
    min_row=min_row,
    max_col=max_col,
    max_row=max_row
)

categories_from_sheet = Reference(
    sheet,
    min_col=min_col,
    min_row=min_row+1,
    max_col=min_col,
    max_row=max_row
)

barchart.add_data(data_from_sheet, titles_from_data=True)
barchart.set_categories(categories_from_sheet)
sheet.add_chart(barchart, "B12")
barchart.title = "Sales by Product line"
barchart.style = 5

wb.save('barchart.xlsx')


Note: you may need to restart the kernel to use updated packages.


In [17]:
from openpyxl.chart.reference import get_column_letter

wb: Workbook = load_workbook('barchart.xlsx')
sheet: Workbook = wb['Report']

min_col: int = wb.active.min_column
max_col: int = wb.active.max_column
min_row: int = wb.active.min_row
max_row: int = wb.active.max_row

for i in range(min_col+1, max_col+1):
    letter = get_column_letter(i)
    sheet[f'{letter}{max_row+1}'] = f'=SUM({letter}{min_row+1}:{letter}{max_row})'
    sheet[f'{letter}{max_row+1}'].style = 'Currency'

wb.save('report.xlsx')